In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,roc_auc_score
from hyperopt import hp,rand,fmin,Trials

In [2]:
#load the dataset
x_breast,y_breast=load_breast_cancer(return_X_y=True)
x=pd.DataFrame(x_breast)
y=pd.Series(y_breast).map({0:1,1:0})
x.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
#Percentage of the target variable
y.value_counts()/len(y)

0    0.627417
1    0.372583
dtype: float64

In [4]:
#splitting the dataset into train and the test set
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [5]:
#detrmine the hyperparameter space
param_grid={
    'n_estimators':hp.uniform('n_estimators',200,2500,100),
    'max_depth':hp.uniform('max_depth',1,10),
    'learning_rate':hp.uniform('learning_rate',0.01,0.99),
    'booster':hp.choice('booster',['gbtree','dart']),
    'gamma':hp.quniform('gamma',0.01,10,0.1),
    'subsample':hp.uniform('subsample',0.50,0.90),
    'colsample_bytree':hp.uniform('colsample_bytree',0.50,0.99),
    'colsample_bylevel':hp.uniform('colsample_bylevel',0.51,0.98),
    'colsample_bynode':hp.uniform('colsample_bynode',0.52,0.97),
    'reg_lambda':hp.uniform('reg_lambda',1,20)
    
}

# Define the objective function

This is the hyperparameter response space,the fuunction we want to minimize

In [6]:
#The objective function takes the hyperparameter as the input
def objective(params):
    param_dict={
        'n_estimators':int(params['n_estimators']),
        'max_depth':int(params['max_depth']),
        'learning_rate':params['learning_rate'],
        'booster':params['booster'],
        'gamma':params['gamma'],
        'subsample':params['subsample'],
        'colsample_bytree':params['colsample_bytree'],
        'colsample_bylevel':params['colsample_bylevel'],
        'colsample_bynode':params['colsample_bynode'],
        'random_state':1000,
    }
    gbm=XGBClassifier(**param_dict)
    #train with cv
    score=cross_val_score(gbm,x_train,y_train,scoring='accuracy',cv=5,n_jobs=4).mean()
    #to minimize we negate the score
    return -score

# Randomized Search

In [7]:
#SEED=np.random.seed(42)

In [8]:
search=fmin(fn=objective,
           space=param_grid,
            max_evals=50,
          rstate=np.random.seed(42),
           algo=rand.suggest,) #randomized search

ERROR in rec_eval                                                                                                      
EXCEPTION                                                                                                              
<class 'TypeError'>                                                                                                    
uniform() got multiple values for argument 'rng'                                                                       
NODE                                                                                                                   
0 uniform                                                                                                              
1   Literal{200}
2   Literal{2500}
3   Literal{100}
4  size =
5   len
6     array_union
7       array_union
8         array_union
9           Literal{new_ids}
10  rng =
11   Literal{rng-placeholder}
  0%|                                                                           |

TypeError: uniform() got multiple values for argument 'rng'